In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image

### Loaded dataset

In [2]:
# Function to load image datasets
def load_image_dataset(image_dir1, image_dir2, image_size=(224, 224)):
    """Loads image datasets from two directories with corresponding labels.

    Args:
        image_dir1: Path to the first directory containing images.
        image_dir2: Path to the second directory containing images.
        image_size: Tuple specifying the desired image size (width, height).

    Returns:
        A tuple containing the image data (NumPy array) and labels (NumPy array).
    """
    images = []
    labels = []

    for filename in os.listdir(image_dir1):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(image_dir1, filename)
            try:
                img = Image.open(filepath).convert("RGB").resize(image_size)
                img_array = np.array(img) / 255.0 # Normalize pixel values
                images.append(img_array)
                labels.append(0) # Label for images from the first directory
            except Exception as e:
                print(f"Error loading {filename}: {e}")

    for filename in os.listdir(image_dir2):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(image_dir2, filename)
            try:
                img = Image.open(filepath).convert("RGB").resize(image_size)
                img_array = np.array(img) / 255.0
                images.append(img_array)
                labels.append(1) # Label for images from the second directory
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                
    return np.array(images), np.array(labels)

# Function to split dataset
def split_dataset(images, labels, test_size=0.2, val_size=0.1, random_state=42):
    """Splits the image dataset into training, testing, and validation sets."""
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=random_state, stratify=labels)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=val_size / (1 - test_size), random_state=random_state, stratify=y_train)
    return x_train, y_train, x_val, y_val, x_test, y_test


In [ ]:
# Example usage (Replace with your actual directories)
image_dir1 = "./cherry_coffee/not_good"
image_dir2 = "./cherry_coffee/good"
images, labels = load_image_dataset(image_dir1, image_dir2)
x_train, y_train, x_val, y_val, x_test, y_test = split_dataset(images, labels)

# Print dataset sizes
print("Training set size:", len(x_train))
print("Validation set size:", len(x_val))
print("Testing set size:", len(x_test))

Training set size: 296
Validation set size: 43
Testing set size: 85


### Training model

In [4]:
# Define the model
def create_model():
    model = keras.Sequential([
        layers.Input(shape=(img_height, img_width, 3)),  # Input shape for RGB images
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    return model

# Define image dimensions
img_height = 224  # Example dimensions, adjust as needed
img_width = 224

# Create the model
model = create_model()

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define a ModelCheckpoint callback
checkpoint = keras.callbacks.ModelCheckpoint("best_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model
model.fit(x_train,
          y_train,
          batch_size=16,
          epochs=10,  # Adjust number of epochs as needed
          validation_data=(x_val, y_val),
          callbacks=[checkpoint])

# Evaluate the model
_, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

2024-11-05 10:24:13.849075: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-11-05 10:24:13.849122: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-05 10:24:13.849135: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-05 10:24:13.849544: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-05 10:24:13.849932: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2024-11-05 10:24:14.570988: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - ETA: 0s - loss: 1.4721 - accuracy: 0.5473
Epoch 1: val_accuracy improved from -inf to 0.69767, saving model to best_model.h5


2024-11-05 10:24:16.235837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/mlp/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 2s 87ms/step - loss: 1.4721 - accuracy: 0.5473 - val_loss: 0.6641 - val_accuracy: 0.6977
Epoch 2/10
18/19 [===========================>..] - ETA: 0s - loss: 0.6322 - accuracy: 0.6806
Epoch 2: val_accuracy did not improve from 0.69767
19/19 [==============================] - 1s 63ms/step - loss: 0.6326 - accuracy: 0.6824 - val_loss: 0.5975 - val_accuracy: 0.6977
Epoch 3/10
18/19 [===========================>..] - ETA: 0s - loss: 0.6348 - accuracy: 0.7083
Epoch 3: val_accuracy did not improve from 0.69767
19/19 [==============================] - 1s 62ms/step - loss: 0.6380 - accuracy: 0.7061 - val_loss: 0.7409 - val_accuracy: 0.6977
Epoch 4/10
18/19 [===========================>..] - ETA: 0s - loss: 0.6185 - accuracy: 0.7014
Epoch 4: val_accuracy did not improve from 0.69767
19/19 [==============================] - 1s 64ms/step - loss: 0.6116 - accuracy: 0.7061 - val_loss: 0.7062 - val_accuracy: 0.6977
Epoch 5/10
18/19 [===========================

**Convert to .tflite**

In [5]:
# Save the model as .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/tmpqe6p7uxm/assets


INFO:tensorflow:Assets written to: /var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/tmpqe6p7uxm/assets
2024-11-05 10:24:32.245424: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-11-05 10:24:32.245606: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-11-05 10:24:32.246355: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/tmpqe6p7uxm
2024-11-05 10:24:32.247252: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-11-05 10:24:32.247257: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/tmpqe6p7uxm
2024-11-05 10:24:32.249498: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-11-05 10:24:32.250427: I tensorflow/cc/saved_model/load

### Test with captured image

In [ ]:
# load image
image_size = (224, 224)
path = "/test.jpg"

# Load, convert, and resize the image
img = Image.open(path).convert("RGB").resize(image_size)
img_array = np.array(img) / 255.0

# Add a batch dimension
img_array = np.expand_dims(img_array, axis=0)
print("image size: ", img_array.shape)

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("input: \n", input_details)
print("output: \n", output_details)

# Function for making predictions
def predict_tflite(image):
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    prediction = np.argmax(output_data)
    return "Class 1" if prediction == 1 else "Class 0"

# Make the prediction
prediction = predict_tflite(img_array.astype(np.float32))
print("Prediction:", prediction)


image size:  (1, 224, 224, 3)
input: 
 [{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
output: 
 [{'name': 'StatefulPartitionedCall:0', 'index': 21, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Prediction: Class 0
